#### Historical Weather Karslruhe

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 49.0076,
	"longitude": 8.4012,
	"start_date": "2012-01-01",
	"end_date": "2025-03-01",
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "weather_code", "cloud_cover", "wind_speed_10m", "shortwave_radiation"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["weather_code"] = hourly_weather_code
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)



Coordinates 49.033390045166016°N 8.432708740234375°E
Elevation 118.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
                            date  temperature_2m  relative_humidity_2m  \
0      2012-01-01 00:00:00+00:00        9.762501             96.372925   
1      2012-01-01 01:00:00+00:00        9.812500             96.699097   
2      2012-01-01 02:00:00+00:00        9.912500             96.053635   
3      2012-01-01 03:00:00+00:00        9.962501             96.055168   
4      2012-01-01 04:00:00+00:00        9.962501             95.732712   
...                          ...             ...                   ...   
115411 2025-03-01 19:00:00+00:00        5.228000             72.985779   
115412 2025-03-01 20:00:00+00:00        5.178000             73.769310   
115413 2025-03-01 21:00:00+00:00        4.128000             76.291832   
115414 2025-03-01 22:00:00+00:00        2.978000             77.780182   
115415 2025-03-01 23:00:00+00:00        2.328000            

In [2]:
historical_weather = hourly_dataframe.copy()

historical_weather.rename(columns={'date': 'Datetime'}, inplace=True)
historical_weather['Datetime'] = pd.to_datetime(historical_weather['Datetime']) 
historical_weather = historical_weather.set_index('Datetime')
historical_weather = historical_weather.tz_convert('Europe/Berlin')

In [3]:
historical_weather.to_csv("historical_weather_karslruhe.csv")